# Decision Tree

# Import libraries

In [2]:
from sklearn import tree
import numpy as np

# Load data

In [20]:
# Load data in to np arrays
hrnames = np.genfromtxt('HR_data.csv', delimiter=",", skip_header = 0, dtype = str)[0]
hrdata = np.genfromtxt('HR_data.csv', delimiter=",", skip_header = 1, dtype = float)

# Delete NaN columns
hrnames = np.delete(hrnames, [7,9], axis=0)
hrnames = hrnames.tolist()
hrdata = np.delete(hrdata, [7,9], axis=1)
hrdata = hrdata.tolist()

# Split out feature and target (input/output)
# hrnames_feature = hrnames[0:-1]
# hrnames_target = hrnames[-1]
# hrdata_feature = hrdata[:,:-1]
# hrdata_target = hrdata[:,-1]

In [19]:
hrdata

[[nan, nan, nan, nan, nan, nan, nan, nan, nan],
 [0.92, 0.87, 4.0, 226.0, 6.0, 1.0, 0.0, 77000.0, 1.0],
 [0.92, 0.87, 4.0, 226.0, 6.0, 1.0, 0.0, 71000.0, 1.0],
 [0.92, 0.87, 4.0, 226.0, 6.0, 1.0, 0.0, 68000.0, 1.0],
 [0.92, 0.82, 5.0, 265.0, 5.0, 0.0, 0.0, 72000.0, 1.0],
 [0.91, 0.97, 5.0, 233.0, 5.0, 0.0, 0.0, 65000.0, 1.0],
 [0.91, 0.97, 5.0, 233.0, 5.0, 0.0, 0.0, 60000.0, 1.0],
 [0.91, 0.97, 5.0, 233.0, 5.0, 0.0, 0.0, 76000.0, 1.0],
 [0.91, 0.95, 5.0, 241.0, 5.0, 0.0, 0.0, 67000.0, 1.0],
 [0.91, 1.0, 4.0, 245.0, 6.0, 0.0, 0.0, 84000.0, 1.0],
 [0.91, 0.9, 4.0, 250.0, 5.0, 0.0, 0.0, 84000.0, 1.0],
 [0.91, 1.0, 4.0, 253.0, 5.0, 1.0, 0.0, 73000.0, 1.0],
 [0.91, 0.99, 5.0, 254.0, 5.0, 0.0, 0.0, 70000.0, 1.0],
 [0.91, 0.99, 5.0, 254.0, 5.0, 0.0, 0.0, 61000.0, 1.0],
 [0.91, 1.0, 4.0, 257.0, 5.0, 0.0, 0.0, 80000.0, 1.0],
 [0.91, 1.0, 4.0, 257.0, 5.0, 0.0, 0.0, 75000.0, 1.0],
 [0.91, 1.0, 4.0, 257.0, 5.0, 0.0, 0.0, 63000.0, 1.0],
 [0.91, 0.84, 5.0, 265.0, 5.0, 0.0, 0.0, 64000.0, 1.0],
 [0.91

# Build and fit decision tree

In [5]:
# Fit decision tree on data
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(hrdata_feature, hrdata_target)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

# Output fitted decision tree

### Function for outputting rules json

In [1]:
def rules(clf, features, labels, node_index=0):
    """Structure of rules in a fit decision tree classifier

    Parameters
    ----------
    clf : DecisionTreeClassifier
        A tree that has already been fit.

    features, labels : lists of str
        The names of the features and labels, respectively.

    """
    node = {}
    if clf.tree_.children_left[node_index] == -1:  # indicates leaf
        count_labels = zip(clf.tree_.value[node_index, 0], labels)
        percent_chance = count_labels[1][0]/(count_labels[1][0]+count_labels[0][0])*100
        n = int(count_labels[0][0] + count_labels[1][0])
        node['name'] = '{}% chance of leaving, n = {}'.format(round(percent_chance,2), n)
    else:
        feature = features[clf.tree_.feature[node_index]]
        threshold = clf.tree_.threshold[node_index]
        node['name'] = '{} > {}'.format(feature, round(threshold, 2))
        left_index = clf.tree_.children_left[node_index]
        right_index = clf.tree_.children_right[node_index]
        node['children'] = [rules(clf, features, labels, right_index),
                            rules(clf, features, labels, left_index)]
    return node

### Use "rules" function and write new file "hrrules.json"

In [10]:
import json

# Generate fitted rules json
r = rules(clf, hrnames_feature, hrnames_target)
with open('hrrules_over.json', 'w') as f:
    f.write(json.dumps(r))